<a href="https://colab.research.google.com/github/rybakinaea/Neuron_DZ/blob/main/%D0%9A%D0%BE%D0%BD%D1%82%D1%80%D0%BE%D0%BB%D1%8C_%D0%B7%D0%BD%D0%B0%D0%BD%D0%B8%D0%B9_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Задание

# 1.

1) Запустить ячейку ниже, чтобы получить 2 жанра. Для полученных жанров провести все этапы предварительной обработки текста (как в примере), обучить наивный байесовский классификатор, численно оценить его работу.

In [1]:
 import random
 lst = ['Christian', 'Country', 'Pop', 'Rock', 'R&B']
 print('ваши жанры', random.choice(lst), 'и', random.choice(lst))

ваши жанры Rock и Christian


In [39]:
import pandas as pd
from pandas.core.common import flatten
from google.colab import drive
drive.mount('/content/drive/')
data = pd.read_csv("/content/dataset.csv")
columns = data[['genre', 'lyrics']]
columns = columns[(columns.genre == 'Christian') | (columns.genre == 'Rock')]

columns['lowered'] = columns['lyrics'].str.lower()
columns

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,genre,lyrics,lowered
0,Christian,"Who am I, that the Lord of all the earth Woul...","who am i, that the lord of all the earth woul..."
1,Christian,Glory Revealed By His Wounds He was pierced ...,glory revealed by his wounds he was pierced ...
2,Christian,Lord of heaven and earth Lord of all creation...,lord of heaven and earth lord of all creation...
3,Christian,I can only imagine what it will be like When ...,i can only imagine what it will be like when ...
4,Christian,I am not skilled to understand What God has w...,i am not skilled to understand what god has w...
...,...,...,...
462,Rock,Cant explain all the feelings that youre maki...,cant explain all the feelings that youre maki...
463,Rock,"One foot on the brake and one on the gas, hey...","one foot on the brake and one on the gas, hey..."
464,Rock,Carry on my wayward son Therell be peace when...,carry on my wayward son therell be peace when...
465,Rock,"Ooh yeah Turn it up Come on Im working hard,...","ooh yeah turn it up come on im working hard,..."


*Токенизация*

In [40]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Удаление стоп-слов

In [41]:
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:
    without_stop.append(", ".join(a))
columns['without_stop'] = without_stop

Лемматизация

In [43]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [44]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:
    lemma.append(", ".join(a))
columns['lemmatized'] = lemma

Разделение данных

In [45]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.genre, train_size = 0.7)
columns.genre.value_counts()

Rock         95
Christian    94
Name: genre, dtype: int64

Векторизация

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

Классификация

In [47]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)
vectorized_x_test = vectorizer.transform(x_test)

Оценка классификации

In [48]:
from sklearn.metrics import *
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

   Christian       0.93      0.93      0.93        29
        Rock       0.93      0.93      0.93        28

    accuracy                           0.93        57
   macro avg       0.93      0.93      0.93        57
weighted avg       0.93      0.93      0.93        57



# 2.

2) Найти (нагуглить) по песне каждого из жанров, которые Вам достались, после необходимой обработки их текстов определить жанр обеих песен с помощью обученной в ходе выполнения предыдущего пункта модели.

In [49]:
#группа evancense
music = ["Don't cry to me. If you loved me. You would be here with me. You want me, come find me. Make up your mind. Should I let you fall. And lose it all?. So maybe you can remember yourself. Can't keep believing. We're only deceiving ourselves. And I'm sick of the lies. And you're too late. Don't cry to me. If you loved me. You would be here with me. You want me, come find me. Make up your mind. Couldn't take the blame. Sick with shame. Must be exhausting to lose your own game. Selfishly hated. No wonder you're jaded. You can't play the victim this time. And you're too late. So don't cry to me. If you loved me. You would be here with me. You want me, come find me. Make up your mind. You never call me when you're sober. You only want it cause it's over. It's over. How could I have burned paradise?. How could I?. You were never mine. So don't cry to me. If you loved me. You would be here with me. Don't lie to me. Just get your things. I've made up your mind"]
#нижний регистр
music_lower = list(map(str.lower, music))
#токенизация
music_tok = [nltk.word_tokenize(i) for i in music_lower]
music_tokened = list(flatten(music_tok))
#удаление стоп-слов
music_withoutstop = list(set(music_tokened) - set(noise))
#лемматизация
music_pro = [lemmatizer.lemmatize(i) for i in music_withoutstop]

#векторизация
vectorized_mus = vectorizer.transform(music)
clf.predict(vectorized_mus)


array(['Rock'], dtype='<U9')

In [50]:
#christian music
music = ["When He rolls up His sleeves He aint just putting on the ritz (Our God is an awesome God) Theres thunder in His footsteps And lightning in His fists (Our God is an awesome God) And the Lord wasnt joking when He kicked em out of Eden It wasnt for no reason that He shed His blood His return is very close and so you better be believing that Our God is an awesome God Our God is an awesome God He reigns from Heaven above With wisdom, power, and love Our God is an awesome "]
#нижний регистр
music_lower = list(map(str.lower, music))
#токенизация
music_tok = [nltk.word_tokenize(i) for i in music_lower]
music_tokened = list(flatten(music_tok))
#удаление стоп-слов
music_withoutstop = list(set(music_tokened) - set(noise))
#лемматизация
music_pro = [lemmatizer.lemmatize(i) for i in music_withoutstop]

#векторизация
vectorized_mus = vectorizer.transform(music)
clf.predict(vectorized_mus)

array(['Christian'], dtype='<U9')

# 3.

3) С помощью набора данных по ссылке аналогичным образом научить модель отличать тексты песен группы Evanescence от текстов песен Пола МакКартни

In [51]:
import pandas as pd
from pandas.core.common import flatten
from google.colab import drive
drive.mount('/content/drive/')
dat = pd.read_csv("/content/dataset-lyrics-musics-mini.csv")
columns = dat[['cantorNome', 'letra']]
columns = columns[(columns.cantorNome == 'evanescence') | (columns.cantorNome == 'paul-mccartney')]

columns['lowered'] = columns['letra'].str.lower()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


*Токенизация*

In [30]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Удаление стоп-слов

In [31]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:
    without_stop.append(", ".join(a))
columns['without_stop'] = without_stop

Лемматизация

In [33]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:
    lemma.append(", ".join(a))
columns['lemmatized'] = lemma

Разделение данных

In [35]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.cantorNome, train_size = 0.7)
columns.cantorNome.value_counts()

paul-mccartney    464
evanescence       154
Name: cantorNome, dtype: int64

Векторизация

In [36]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

Классификация

In [37]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)
vectorized_x_test = vectorizer.transform(x_test)

Оценка классификации

In [38]:
from sklearn.metrics import *
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

                precision    recall  f1-score   support

   evanescence       0.92      0.58      0.71        38
paul-mccartney       0.90      0.99      0.94       148

      accuracy                           0.90       186
     macro avg       0.91      0.78      0.83       186
  weighted avg       0.90      0.90      0.89       186

